In [28]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("sumn2u/garbage-classification-v2")

print("Path to dataset files:", path)

Path to dataset files: /home/pasha/.cache/kagglehub/datasets/sumn2u/garbage-classification-v2/versions/8


In [29]:
import os
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

classes = os.listdir(os.path.join(path, "garbage-dataset"))
print(classes)

['trash', 'glass', 'battery', 'clothes', 'metal', 'plastic', 'cardboard', 'paper', 'biological', 'shoes']


## Извлечение фичей

In [30]:
def calculate_yellow_amount(image):
    """Calculate the amount of yellow color in the image."""
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    lower_yellow = np.array([20, 100, 100])
    upper_yellow = np.array([30, 255, 255])
    yellow_mask = cv2.inRange(hsv_image, lower_yellow, upper_yellow)
    yellow_amount = cv2.countNonZero(yellow_mask)
    return yellow_amount

def calculate_silver_amount(image):
    """Calculate the amount of silver color in the image."""
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    lower_silver = np.array([0, 0, 200])
    upper_silver = np.array([180, 25, 255])
    silver_mask = cv2.inRange(hsv_image, lower_silver, upper_silver)
    silver_amount = cv2.countNonZero(silver_mask)
    return silver_amount

def calculate_parallel_lines(image):
    """Calculate the amount of parallel lines in the image."""
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray_image, 50, 150, apertureSize=3)
    
    # Detect lines using Hough Transform
    lines = cv2.HoughLines(edges, 1, np.pi / 180, threshold=100)
    
    if lines is not None:
        return len(lines)  # Return the number of detected lines
    return 0  # No lines detected

def detect_cylinders(image):
    """Detect cylindrical shapes in the image."""
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred_image = cv2.GaussianBlur(gray_image, (9, 9), 2)

    # Detect circles using Hough Transform
    circles = cv2.HoughCircles(blurred_image, cv2.HOUGH_GRADIENT, dp=1, minDist=30,
                               param1=100, param2=30, minRadius=10, maxRadius=100)

    if circles is not None:
        return circles.shape[1]  # Return the number of detected circles
    return 0  # No circles detected

def get_features(image):
    """Get a set of features (yellow amount, silver amount, parallel lines, cylindrical shapes) from the image."""
    yellow_amount = calculate_yellow_amount(image)
    silver_amount = calculate_silver_amount(image)
    parallel_lines = calculate_parallel_lines(image)
    cylinder_count = detect_cylinders(image)  # New feature for cylindrical shapes
    return yellow_amount, silver_amount, parallel_lines, cylinder_count

In [31]:
test_im = cv2.imread(path + "/garbage-dataset/cardboard/cardboard_25.jpg")
print(get_features(test_im))

(15834, 284114, 140, 15)


## Обучение классификатора

In [32]:
# Define the path to the dataset
base_path = path + "/garbage-dataset"
classes = ['trash', 'glass', 'battery', 'clothes', 'metal', 'plastic', 'cardboard', 'paper', 'biological', 'shoes']

# Initialize lists to hold features and labels
features = []
labels = []

# Loop through each class and extract features from images
for class_name in classes:
    class_path = os.path.join(base_path, class_name)
    
    for image_name in os.listdir(class_path):
        image_path = os.path.join(class_path, image_name)
        
        # Read the image
        image = cv2.imread(image_path)
        
        if image is not None:
            # Use the new functions to calculate yellow and silver amounts, parallel lines, and cylinder count
            yellow_amount, silver_amount, parallel_lines, cylinder_count = get_features(image)
            features.append([yellow_amount, silver_amount, parallel_lines, cylinder_count])
            labels.append(class_name)

# Convert features and labels to a DataFrame
df = pd.DataFrame(features, columns=['yellow_amount', 'silver_amount', 'parallel_lines', 'cylinder_count'])
df['label'] = labels

# Split the dataset into training and testing sets
X = df[['yellow_amount', 'silver_amount', 'parallel_lines', 'cylinder_count']]
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest Classifier
classifier = RandomForestClassifier(n_estimators=100, random_state=42)
classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = classifier.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Function to classify a new image
def classify_image(image_path):
    image = cv2.imread(image_path)
    if image is not None:
        # Use the new functions to calculate yellow and silver amounts, parallel lines, and cylinder count
        yellow_amount, silver_amount, parallel_lines, cylinder_count = get_features(image)
        feature = np.array([[yellow_amount, silver_amount, parallel_lines, cylinder_count]])
        prediction = classifier.predict(feature)
        return prediction[0]
    else:
        return None


Accuracy: 0.3898305084745763
              precision    recall  f1-score   support

     battery       0.31      0.29      0.30       192
  biological       0.36      0.39      0.38       190
   cardboard       0.43      0.44      0.44       386
     clothes       0.54      0.69      0.60      1086
       glass       0.29      0.29      0.29       617
       metal       0.18      0.14      0.16       191
       paper       0.33      0.25      0.28       343
     plastic       0.24      0.18      0.20       392
       shoes       0.27      0.22      0.24       392
       trash       0.27      0.27      0.27       164

    accuracy                           0.39      3953
   macro avg       0.32      0.32      0.32      3953
weighted avg       0.37      0.39      0.37      3953



In [43]:
result = classify_image(path + '/garbage-dataset/glass/glass_686.jpg')  # Adjust the path to a specific image
print("Predicted class:", result)

Predicted class: glass


/home/pasha/image_classification/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


## Проверка гипотез